# Import schema and data to Weaviate

In [20]:
import json
import pandas as pd
import helper

## Initialize the client

In [11]:
import weaviate
client = weaviate.Client("http://localhost:8080")

## Upload schema

In [12]:
client.schema.delete_all()
schema = 'schema.json'
client.schema.create(schema)

## Upload data

We have to upload data step by step. The data objects of references need to exist before you can make a reference to this object. There are two approaches here, but since we have the whole dataset at hand here, we can first add all single data objects, then add data objects with references, like Province and Wine. Let's do it in the following order: 
1. Add all countries
2. Add all provinces, with references to countries
3. Add all wine varieties
4. Add all wines, with references to provinces, countries and varieties.

#### First, let's check how the data looks again 


In [56]:
df = pd.read_csv('data/preprocessed_data.csv', index_col=0)
df.head(3)

,country,description,designation,points,price,province,title,variety
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris


#### Add countries

In [25]:
def add_countries():
    batch = weaviate.ThingsBatchRequest()
    no_items_in_batch = 0

    # get all the unique country names
    country_names = df.country.dropna().unique()

    # loop through the unique country names and add them to the batch
    for country_name in country_names:
        uuid = helper.generate_uuid('Country', country_name)

        country_object = {
            "name": country_name
        }

        batch.add_thing(country_object, "Country", uuid)
    
    client.batch.create_things(batch)

add_countries()

In [50]:
def add_provinces():
    batch = weaviate.ThingsBatchRequest()
    refbatch = weaviate.ReferenceBatchRequest()

    # get all the unique province names with the countries
    group = df.groupby('province')
    df2 = group.apply(lambda x: x['country'].unique())
    df2 = df2.apply(pd.Series)
    df2.reset_index(inplace=True)
    df2.columns = ['province', 'country']
    df2.dropna(inplace=True)
    
    # province_names = df2.province.dropna().unique()
    # country_names = df2.country.dropna().unique()

    # loop through the unique provinces, get country reference and add to batch
    for index, row in df2.iterrows():
        province_name = row['province']
        country_name = row['country']
        
        province_uuid = helper.generate_uuid('Province', province_name)
        country_uuid = helper.generate_uuid('Country', country_name)
        
        province_object = {
            "name": province_name
        }

        batch.add_thing(province_object, "Province", province_uuid)
        refbatch.add_reference(province_uuid, "Province", "inCountry", country_uuid)
    
    client.batch.create_things(batch)
    client.batch.add_references(refbatch)

add_provinces()

In [51]:
def add_provinces():
    batch = weaviate.ThingsBatchRequest()
    refbatch = weaviate.ReferenceBatchRequest()

    # get all the unique province names with the countries
    group = df.groupby('province')
    df2 = group.apply(lambda x: x['country'].unique())
    df2 = df2.apply(pd.Series)
    df2.reset_index(inplace=True)
    df2.columns = ['province', 'country']
    df2.dropna(inplace=True)
    
    # loop through the unique provinces, get country reference and add to batch
    for index, row in df2.iterrows():
        province_name = row['province']
        country_name = row['country']
        
        province_uuid = helper.generate_uuid('Province', province_name)
        country_uuid = helper.generate_uuid('Country', country_name)
        
        province_object = {
            "name": province_name
        }

        batch.add_thing(province_object, "Province", province_uuid)
        refbatch.add_reference(province_uuid, "Province", "inCountry", country_uuid)
    
    client.batch.create_things(batch)
    client.batch.add_references(refbatch)

add_provinces()

In [55]:
def add_varieties():
    batch = weaviate.ThingsBatchRequest()

    # get all the unique variety names
    variety_names = df.variety.dropna().unique()

    # loop through the unique country names and add them to the batch
    for variety_name in variety_names:
        uuid = helper.generate_uuid('Variety', variety_name)

        variety_object = {
            "name": variety_name
        }

        batch.add_thing(variety_object, "Variety", uuid)
    
    client.batch.create_things(batch)

add_varieties()

In [57]:
df.head(3)

,country,description,designation,points,price,province,title,variety
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris


In [89]:
def add_wines(max_items=10000000):
    batch_size = 512
    batch = weaviate.ThingsBatchRequest()
    no_items_in_batch = 0
    refbatch = weaviate.ReferenceBatchRequest()
    no_items_in_refbatch = 0

    df2 = df[df['title'].notna()]
    df2 = df2[df2['description'].notna()]
    
    imported_items = 0
    for index, row in df2.iterrows():
        if imported_items > max_items:
            client.batch.create_things(batch)
            client.batch.add_references(refbatch)
            return
        
        wine_object = {
            "title": row["title"],
            "description": row["description"],
        }
        
        if not pd.isnull(row['designation']):
            wine_object['designation'] = row['designation']
        if not pd.isnull(row['points']):
            wine_object['points'] = int(row['points'])
        if not pd.isnull(row['price']):
            wine_object['price'] = float(row['price'])
        
        
        wine_uuid = helper.generate_uuid('wine', row["title"]+row["description"])
            
        batch.add_thing(wine_object, "Wine", wine_uuid)
        no_items_in_batch += 1
        
        # add ref to country
        if not pd.isnull(row['country']):
            country_name = row['country']
            country_uuid = helper.generate_uuid('Country', country_name)
            refbatch.add_reference(wine_uuid, "Wine", "fromCountry", country_uuid)
            no_items_in_refbatch += 1

        # add ref to province
        if not pd.isnull(row['province']):
            province_name = row['province']
            province_uuid = helper.generate_uuid('Province', province_name)
            refbatch.add_reference(wine_uuid, "Wine", "fromProvince", province_uuid)
            no_items_in_refbatch += 1

        # add ref to variety
        if not pd.isnull(row['variety']):
            variety_name = row['variety']
            variety_uuid = helper.generate_uuid('Variety', variety_name)
            refbatch.add_reference(wine_uuid, "Wine", "hasVariety", variety_uuid)
            no_items_in_refbatch += 1

        if no_items_in_batch >= batch_size:
            results = client.batch.create_things(batch)
            # for result in results:
            #     if result['result'] != {}:
            #         helper.log(result['result'])

            batch = weaviate.ThingsBatchRequest()
            no_items_in_batch = 0
            imported_items += no_items_in_batch

        if no_items_in_refbatch >= batch_size:
            results = client.batch.add_references(refbatch)
            # for result in results:
            #     if result['result']['status'] != 'SUCCESS':
            #         helper.log(result['result'])

            refbatch = weaviate.ReferenceBatchRequest()
            no_items_in_refbatch = 0
            
    client.batch.create_things(batch)
    client.batch.add_references(refbatch)

add_wines(200)

KeyboardInterrupt: 